In [1]:
import pandas as pd

# Sample user-item rating matrix
data = {
    'User': ['A', 'A', 'A', 'B', 'B', 'C', 'C', 'D'],
    'Item': ['Movie1', 'Movie2', 'Movie3', 'Movie1', 'Movie3', 'Movie2', 'Movie4', 'Movie4'],
    'Rating': [5, 3, 4, 4, 5, 2, 5, 4]
}

df = pd.DataFrame(data)
df

,User,Item,Rating
0,A,Movie1,5
1,A,Movie2,3
2,A,Movie3,4
3,B,Movie1,4
4,B,Movie3,5
5,C,Movie2,2
6,C,Movie4,5
7,D,Movie4,4


In [2]:
user_item_matrix = df.pivot_table(
    index='User',
    columns='Item',
    values='Rating'
).fillna(0)

user_item_matrix


Item,Movie1,Movie2,Movie3,Movie4
User,,,,
A,5.0,3.0,4.0,0.0
B,4.0,0.0,5.0,0.0
C,0.0,2.0,0.0,5.0
D,0.0,0.0,0.0,4.0


In [3]:
from sklearn.metrics.pairwise import cosine_similarity

user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(
    user_similarity,
    index=user_item_matrix.index,
    columns=user_item_matrix.index
)

user_similarity_df

User,A,B,C,D
User,,,,
A,1.000000,0.883452,0.157568,0.000000
B,0.883452,1.000000,0.000000,0.000000
C,0.157568,0.000000,1.000000,0.928477
D,0.000000,0.000000,0.928477,1.000000


In [4]:
def get_similar_users(user, similarity_df, n=2):
    return similarity_df[user].sort_values(ascending=False)[1:n+1]

get_similar_users('A', user_similarity_df)

User
B    0.883452
C    0.157568
Name: A, dtype: float64

In [5]:
def recommend_items(user, user_item_matrix, similarity_df, n_recommendations=2):
    similar_users = get_similar_users(user, similarity_df)

    weighted_scores = user_item_matrix.loc[similar_users.index].T.dot(similar_users)
    already_rated = user_item_matrix.loc[user]

    recommendations = weighted_scores[already_rated == 0]
    return recommendations.sort_values(ascending=False).head(n_recommendations)

recommend_items('A', user_item_matrix, user_similarity_df)

Item
Movie4    0.787839
dtype: float64